In [4]:
# import libraries
import os
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interpn
from scipy.interpolate import interp1d
import pickle 
import xarray as xr
import matplotlib.colors as colors
import seaborn as sns
import cmocean

# import some home-made functions
import nbimporter

# for mapping
import cartopy.crs as ccrs
# https://cosima-recipes.readthedocs.io/en/latest/tutorials/Making_Maps_with_Cartopy.html
import cartopy.feature as cft
# BIG issue with cartopy: see here https://github.com/SciTools/cartopy/issues/1325 at @jypeter
# I had to download a bunch of stuff myself and put it in ~/.local/share/cartopy/shapefile/.../...
# https://stackoverflow.com/questions/30030328/correct-placement-of-colorbar-relative-to-geo-axes-cartopy
# https://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html
land_110m = cft.NaturalEarthFeature('physical', 'land', '110m', edgecolor='black', facecolor='gray', linewidth=0.5) # continents

In [8]:
data = open("../data/output_data_for_figures/data_fig2.pkl", "rb")
data = pickle.load(data)

In [9]:
weighted_mean_f100 = data['weighted_mean_f100']
lonT = data['lon']
latT = data['lat']
mask = data['mask']
depthT = data['depthT']
f100 = data['f100']
f100_weighted_lines = data['f100_weighted_lines']


In [10]:
# mean (unique) f100 profiles
f100_global_average = np.nanmean(np.nanmean(f100, axis = 0), axis = 0)
df = pd.DataFrame({'depth':depthT, 'f100': f100_global_average})

<ipython-input-10-f94cf05764d8>:2: RuntimeWarning: Mean of empty slice
  f100_global_average = np.nanmean(np.nanmean(f100, axis = 0), axis = 0)


In [12]:
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/gridspec_multicolumn.html

deep_pixel_mask = mask[:,:,np.newaxis]
pal = 'turbo'

from matplotlib.gridspec import GridSpec
fig = plt.figure(constrained_layout=True, figsize = (12,8))
gs = GridSpec(3, 3, figure=fig)

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.Robinson())
ax1.coastlines(resolution='110m', color='black', linestyle='-', alpha=1, linewidth = 2)
p1 = plt.pcolormesh(lonT, latT, weighted_mean_f100, vmin = 0, vmax = 1, shading = 'auto', transform=ccrs.PlateCarree(), cmap = pal)
ax1.add_feature(land_110m)
ax1.contour(lonT, latT, f100_weighted_lines, vmin = np.nanmedian(weighted_mean_f100),
                 transform = ccrs.PlateCarree(), colors = 'white', linewidths = 1)
ax1.set_title('Water-column average f$_{100}$', fontsize = 20)

# fig2
ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.Robinson())
ax2.coastlines(resolution='110m', color='black', linestyle='-', alpha=1, linewidth = 2)
p2 = ax2.pcolormesh(lonT, latT, (f100[:,:,19]*deep_pixel_mask[:,:,0]), vmin = 0, vmax = 1, shading = 'auto', transform=ccrs.PlateCarree(), cmap = pal) 
d = int(np.floor(depthT[19]))
ax2.add_feature(land_110m)
ax2.set_title(str(d)+ ' m', fontsize = 16) 

# fig3
ax3 = fig.add_subplot(gs[1, 1], projection=ccrs.Robinson())
ax3.coastlines(resolution='110m', color='black', linestyle='-', alpha=1, linewidth = 2)
p3 = ax3.pcolormesh(lonT, latT, (f100[:,:,25]*deep_pixel_mask[:,:,0]), vmin = 0, vmax = 1, shading = 'auto', transform=ccrs.PlateCarree(), cmap = pal)
d = int(np.ceil(depthT[25]))
ax3.add_feature(land_110m)
ax3.set_title(str(d)+ ' m', fontsize = 16) 

# fig4
ax4 = fig.add_subplot(gs[2, 1], projection=ccrs.Robinson())
ax4.coastlines(resolution='110m', color='black', linestyle='-', alpha=1, linewidth = 2)
p4 = ax4.pcolormesh(lonT, latT, (f100[:,:,33]*deep_pixel_mask[:,:,0]), vmin = 0, vmax = 1, shading = 'auto', transform=ccrs.PlateCarree(), cmap = pal)
d = int(np.ceil(depthT[33]))
ax4.add_feature(land_110m)
ax4.set_title(str(d)+ ' m', fontsize = 16)

ax5 = fig.add_subplot(gs[1:, 0])
ax5 = plt.scatter(np.array(df['f100']),np.array(df['depth']), linewidth = 2, color = 'black')
ax5 = plt.xlabel('f$_{100}$', fontsize = 16)
ax5 = plt.ylabel('Depth [m]', fontsize = 16)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=12)
# plt.gca().set_title('Mean f$_{100}$ profile')

cbar_ax = fig.add_axes([.70, 0.20, 0.02, 0.64])
col = fig.colorbar(p4, cax=cbar_ax)
plt.savefig('Fig2.jpg', bbox_inches = "tight", dpi=300)